# 2019 H1B Visa Petitions Analysis 

# What is H1B Visa?
##### In the United States, H1B is one of the Non-immigrant visas. It allows foreign workers to work in the States temporarily in specialty occupations such as IT, medical or business industries. It also requires workers to have a bachelor's or higher degree. Until 2020, more than 580K people are working under H1B in the USA. Upon the current immigration laws, the United States Citizenship and Immigration Services issues 65K H1B visas per year. 

# Contents

- Overview of Top 30 Job Titles With Most Filed H1B Cases
- Number of IT Job H1B Cases by States
- Top 10 Sponsors With Most Filed H1B Cases (IT Job ONLY)
- Number of Top 12 IT Job Employment Comparison : H1B vs National 
- Avg Salary Comparison: H1B vs National 
- H1B Case Status Percentages 
- Deneid H1B Cases
    - Salary vs Job titles 
    - 5 Numbers Summary: Certified vs Denied 
- Certified Rate vs Avg Salary : Top 10 Sponsors
- Relationship between Salary and Certified Rate

# Data Sources

- U.S Department of Labor
    - https://www.dol.gov/agencies/eta/foreign-labor/performance
- U.S Bureau of Labor Statistics
    - https://www.bls.gov/oes/current/oes_nat.htm
    
You also can find datasets here:
- https://drive.google.com/drive/folders/1r1N8d-YvC0UdR9HIhhxgHhs7ZBoPzl4C?usp=sharing


# Challenges of Datasets
- Datapoints: More than 660K
- Most of the columns have missing values
- Non-Normalized Employer Names
- Each job code matches multiple job titles
- Wrong information in various columns, such as the data in the column represents the unit of wages
- Inconsistent States'names (Mixed of Full state name & abbreviation)
- Inconsistent wages information (Wage units: Year, Month, Week, Bi-Weekly, Hour)

# Data Preprocessing

### Import packages

In [76]:
import pandas as pd
import plotly.offline as pyo
import plotly.io as pio
import chart_studio
#import chart_studio.plotly as py
import plotly.express as px
import fnmatch
import numpy as np
import plotly.graph_objects as go
from cleanco import cleanco
from plotly.subplots import make_subplots
import fuzzy_pandas as fpd 
import plotly

### Load files

In [74]:
#py.sign_in('jiachenS', 'DwaaT9qARyVOTnBMYLM4')
chart_studio.tools.set_credentials_file(username='jiachenS', api_key='DwaaT9qARyVOTnBMYLM4')

In [4]:
#Because file has too many columns, we only load data with needed columns
fields = ['CASE_STATUS','VISA_CLASS','JOB_TITLE', 'SOC_CODE','SOC_TITLE',
         #'TOTAL_WORKER_POSITIONS', 'NEW_EMPLOYMENT','CONTINUED_EMPLOYMENT',
        #  'NEW_CONCURRENT_EMPLOYMENT','CHANGE_EMPLOYER',
          'EMPLOYER_NAME','WORKSITE_STATE_1',
         'PREVAILING_WAGE_1','PW_UNIT_OF_PAY_1','FULL_TIME_POSITION']

In [5]:
#load file, the data has 260 columns, here we just load columns as needed. 
df = pd.read_csv('H-1B_Disclosure_Data_FY2019.csv',usecols=fields,dtype={'CASE_STATUS':'string',
                                                                         'VISA_CLASS':'string',
                                                                         'JOB_TITLE':'string',
                                                                        # 'TOTAL_WORKER_POISTIONS':int,
                                                                         #'NEW_EMPLOYMENT':'string',
                                                                          #'CONTINUED_EMPLOYMENT':'string',
                                                                         #'NEW_CONCURRENT_EMPLOYMENT':'string',
                                                                         #'CHANGE_EMPLOYER':'string',
                                                                         'SOC_TITLE':'string',
                                                                        'SOC_CODE':'string',
                                                                         'EMPLOYER_NAME':'string',
                                                                         #'WORKSITE_CITY_1':'string',
                                                                         'WORKSITE_STATE_1':'string',
                                                                         "PREVAILING_WAGE_1":float,
                                                                         "PW_UNIT_OF_PAY_1":'string',
                                                                        'FULL_TIME_POSITION':'string'})

In [6]:
#for this project, we only need data of visa type 'H1B' 
df1=df[(df.VISA_CLASS=='H-1B')]

In [7]:
#check the first five rows of df1
df1.head()

,CASE_STATUS,VISA_CLASS,JOB_TITLE,SOC_CODE,SOC_TITLE,FULL_TIME_POSITION,EMPLOYER_NAME,WORKSITE_STATE_1,PREVAILING_WAGE_1,PW_UNIT_OF_PAY_1
0,WITHDRAWN,H-1B,ASSOCIATE CREATIVE DIRECTOR,40848,ADVERTISING AND PROMOTIONS MANAGERS,Y,"R/GA MEDIA GROUP, INC.",NY,NaN,<NA>
5,WITHDRAWN,H-1B,GLOBAL BRAND SUPERVISOR,40848,ADVERTISING AND PROMOTIONS MANAGERS,Y,"MCCANN-ERICKSON MARKETING, INC.",GA,NaN,<NA>
6,WITHDRAWN,H-1B,DIRECTOR OF BUSINESS DEVELOPMENT,40848,ADVERTISING AND PROMOTIONS MANAGERS,Y,MH SUB I LLC,CA,97365.0,Year
7,WITHDRAWN,H-1B,HEAD OF BUSINESS DEVELOPMENT,40848,ADVERTISING AND PROMOTIONS MANAGERS,Y,"KIIP, INC.",CALIFORNIA,131685.0,Year
8,WITHDRAWN,H-1B,"ACCOUNT DIRECTOR, SOCIAL MEDIA",40848,ADVERTISING AND PROMOTIONS MANAGERS,Y,"SUNSHINE SACHS & ASSOCIATES WEST, LLC",CALIFORNIA,97365.0,Year


### Check Missing Values

In [8]:
#check missing values in each column 
df1.isnull().sum(axis=0)

CASE_STATUS               0
VISA_CLASS                0
JOB_TITLE                 1
SOC_CODE                  4
SOC_TITLE                 4
FULL_TIME_POSITION        0
EMPLOYER_NAME            10
WORKSITE_STATE_1         16
PREVAILING_WAGE_1     32477
PW_UNIT_OF_PAY_1      32477
dtype: int64

In [9]:
#drop rows with missing values in columns 'job title' and 'soc code'
df2 = df1.dropna(subset = ['JOB_TITLE','SOC_CODE'])
#check missing values again 
df2.isnull().sum(axis=0)

CASE_STATUS               0
VISA_CLASS                0
JOB_TITLE                 0
SOC_CODE                  0
SOC_TITLE                 0
FULL_TIME_POSITION        0
EMPLOYER_NAME            10
WORKSITE_STATE_1         16
PREVAILING_WAGE_1     32477
PW_UNIT_OF_PAY_1      32477
dtype: int64

### Create Data Frames 

In [10]:
#create a dataframe for "H1B count by job titles"
#count soc code and get the top 50 soc code 
soc = df2['SOC_CODE'].value_counts()
soc = pd.DataFrame(soc)
soc.reset_index(inplace=True)
soc= soc.rename(columns={'index':'SOC_CODE','SOC_CODE':'COUNT'})
#get top 30 jobs that have most H1B applications
soc30 = soc.nlargest(30,['COUNT'])
soc30.head()

,SOC_CODE,COUNT
0,15-1132,215171
1,15-1121,72636
2,15-1199,63095
3,15-1133,31440
4,15-1131,17270


In [11]:
#df2.iloc[:,5:7]
#df2

In [12]:
#get dataframe for job titles
title = df2.iloc[:,3:5]
title = pd.DataFrame(title)
#drop duplicates
title = title.drop_duplicates()
title.head()

,SOC_CODE,SOC_TITLE
0,40848,ADVERTISING AND PROMOTIONS MANAGERS
376,40848,ACCOUNTANTS AND AUDITORS
377,44501,MARKETING MANAGERS
382,44501,MARKETING MANAGER
2561,44501,GENERAL & OPERATIONS MANAGER


In [13]:
#the below code is for the first graph-Top 30 Job titles
#Add job titles to top 30 soc code
H1B_By_Titles = pd.merge(soc30,title,on = 'SOC_CODE',how = 'inner')
#H1B_By_Titles['SOC_CODE'].value_counts()
#a= H1B_By_Titles[H1B_By_Titles['SOC_CODE']=='15-1132']
H1B_By_Titles.drop_duplicates(subset = ['SOC_CODE'],keep = 'first', inplace = True)
H1B_By_Titles

,SOC_CODE,COUNT,SOC_TITLE
0,15-1132,215171,"SOFTWARE DEVELOPERS, APPLICATIONS"
37,15-1121,72636,COMPUTER SYSTEMS ANALYSTS
43,15-1199,63095,"COMPUTER OCCUPATIONS, ALL OTHER"
86,15-1133,31440,"SOFTWARE DEVELOPERS, SYSTEMS SOFTWARE"
101,15-1131,17270,COMPUTER PROGRAMMERS
104,15-2031,11762,OPERATIONS RESEARCH ANALYSTS
107,17-2141,11549,MECHANICAL ENGINEER
114,13-2011,10574,ACCOUNTANTS AND AUDITORS
120,13-1111,10092,MANAGEMENT ANALYSTS
125,15-2041,9541,STATISTICIANS


In [14]:
#H1B_By_Titles = pd.merge(soc30,title,on = 'SOC_CODE',how = 'inner')

In [15]:
#the below code is for the graph-IT job map
#get data of IT jobs from the previous dataframe by using filter
IT_code = fnmatch.filter(df2['SOC_CODE'].unique(),'15-*')
#create a new dataframe only contains IT jobs
search_code = '|'.join(IT_code)
df3 = df2[df2['SOC_CODE'].str.contains(search_code)]
#check missing values in df3
#df3.isnull().sum(axis=0)
#drop missing values in df3 
df3 = df3.dropna(subset = ['WORKSITE_STATE_1'])
#check missing values again to make we dropped the correct data
df3.isnull().sum(axis=0)

CASE_STATUS               0
VISA_CLASS                0
JOB_TITLE                 0
SOC_CODE                  0
SOC_TITLE                 0
FULL_TIME_POSITION        0
EMPLOYER_NAME             8
WORKSITE_STATE_1          0
PREVAILING_WAGE_1     21063
PW_UNIT_OF_PAY_1      21063
dtype: int64

In [16]:
#check state and city spelling of df3
#state  = df3['WORKSITE_STATE_1'].tolist()
#get the unique values from the list state
#state = set(state)
#state
#US states dictionary
us_state_abbrev = {
    'Alabama': 'AL','Alaska': 'AK','American Samoa': 'AS','Arizona': 'AZ','Arkansas': 'AR',
    'California': 'CA','Colorado': 'CO', 'Connecticut': 'CT','Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA', 'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME','Maryland': 'MD','Massachusetts': 'MA','Michigan': 'MI','Minnesota': 'MN','Mississippi': 'MS',
    'Missouri': 'MO','Montana': 'MT',
    'Nebraska': 'NE','Nevada': 'NV','New Hampshire': 'NH','New Jersey': 'NJ','New Mexico': 'NM',
    'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Northern Mariana Islands':'MP',
    'Ohio': 'OH', 'Oklahoma': 'OK','Oregon': 'OR',
    'Pennsylvania': 'PA', 'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',  'South Dakota': 'SD',
    'Tennessee': 'TN',  'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT', 'Virgin Islands': 'VI', 'Virginia': 'VA',
    'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

#covert dict to a dataframe
US_State = pd.DataFrame(list(us_state_abbrev.items()))
US_State = US_State.rename(columns={0:"WORKSITE_STATE_1",1:"Abb"}) 
US_State['WORKSITE_STATE_1'] = US_State['WORKSITE_STATE_1'].str.upper()
US_State.head()



,WORKSITE_STATE_1,Abb
0,ALABAMA,AL
1,ALASKA,AK
2,AMERICAN SAMOA,AS
3,ARIZONA,AZ
4,ARKANSAS,AR


In [17]:
#get state name as a new column into dataframe 
df4 = pd.merge(df3,US_State,on='WORKSITE_STATE_1',how='left')
#get state name consistent 
df4['Abb'] = np.where(df4['Abb'].isnull(),df4['WORKSITE_STATE_1'],df4['Abb'])
#drop the orginal column of state 
df4 = df4.drop(['WORKSITE_STATE_1'],axis=1)


df5 = df4.groupby('Abb')['VISA_CLASS'].count().to_frame()
df5.reset_index(inplace=True)
df5.rename(columns={'Abb':'state','VISA_CLASS':'count'},inplace=True)
df5.head()

,state,count
0,AK,21
1,AL,873
2,AR,3704
3,AZ,8852
4,CA,86519


In [18]:
#create a dataframe for top 20 companies 
df6 = df2[df2['SOC_CODE'].str.contains(search_code)]
#drop rows with no employer name (8 rows)
df6 = df6.dropna(subset = ['EMPLOYER_NAME'])
#check the missing value for column employer
df6.isnull().sum(axis=0)

CASE_STATUS               0
VISA_CLASS                0
JOB_TITLE                 0
SOC_CODE                  0
SOC_TITLE                 0
FULL_TIME_POSITION        0
EMPLOYER_NAME             0
WORKSITE_STATE_1          6
PREVAILING_WAGE_1     21064
PW_UNIT_OF_PAY_1      21064
dtype: int64

In [19]:
#count employer names - find top 10 sponsors with most H1B petitions
count_emp = pd.DataFrame(df6.groupby('EMPLOYER_NAME')['EMPLOYER_NAME'].count())
count_emp.rename(columns={'EMPLOYER_NAME':'H1B_Count'},inplace=True)
count_emp.reset_index(inplace=True)
count_emp.sort_values(by=['H1B_Count'],ascending=False,inplace=True)
#get top 10 H1B sponsors 
count_emp10 = count_emp.nlargest(10,['H1B_Count'])

#remove commas 
count_emp10.EMPLOYER_NAME = count_emp10.EMPLOYER_NAME.str.replace(',','')
#remove dot
count_emp10.EMPLOYER_NAME= count_emp10.EMPLOYER_NAME.str.replace('.','')
#remove text in parenthesis
count_emp10.EMPLOYER_NAME = count_emp10.EMPLOYER_NAME.str.replace(r"\(.*\)","")
#remove spaces
count_emp10.EMPLOYER_NAME= count_emp10.EMPLOYER_NAME.str.strip()
#remove company legal extensions
count_emp10.EMPLOYER_NAME = count_emp10.EMPLOYER_NAME.apply(lambda x: cleanco(x).clean_name() if type(x)==str else x)
#remove country name
c = ['US','AMERICA']
for cc in c:
    count_emp10.EMPLOYER_NAME = count_emp10.EMPLOYER_NAME.apply(lambda x:x.replace(cc,'')
                                                               if (type(x)==str and x.endswith(cc)) else x)
    
count_emp10

,EMPLOYER_NAME,H1B_Count
5902,COGNIZANT TECHNOLOGY SOLUTIONS,27577
13242,INFOSYS,20886
25590,TATA CONSULTANCY SERVICES,11772
11290,GOOGLE,7792
4636,CAPGEMINI,7590
7590,DELOITTE CONSULTING,6964
1498,AMAZONCOM SERVICES,5465
413,ACCENTURE,5294
16955,MICROSOFT,5076
12608,IBM,4866


In [20]:
#Normalize Employer Names for IT jobs 
#get unique Employer name out of df6
emp_name = df6['EMPLOYER_NAME'].unique()
#convert to a df and assign a name to the column 
emp_name = pd.DataFrame(emp_name)
emp_name.columns = ['Emp_name']
emp_name.head()

,Emp_name
0,"NATERA, INC."
1,AMERICAN EXPRESS TRAVEL RELATED SERVICES COMPA...
2,KEYME INC.
3,CHOICE HOTELS INTERNATIONAL SERVICES CORP
4,SEI INVESTMENTS COMPANY


In [21]:
#fuzzy match - looking for similar names for the top 10 companies, and normalized companies' names. 
match = fpd.fuzzy_merge(count_emp10,emp_name,
                       left_on=['EMPLOYER_NAME'],
                       right_on=['Emp_name'],
                       ignore_case = True,
                       keep='match',
                       method='jaro',
                       threshold=0.73)
match

,EMPLOYER_NAME,Emp_name
0,COGNIZANT TECHNOLOGY SOLUTIONS,COGNIZANT TECHNOLOGY SOLUTIONS U.S. CORPORATION
1,COGNIZANT TECHNOLOGY SOLUTIONS,DB GLOBAL TECHNOLOGY INC.
2,COGNIZANT TECHNOLOGY SOLUTIONS,CLOUDMINDS TECHNOLOGY INC.
3,COGNIZANT TECHNOLOGY SOLUTIONS,CONSOLE TECHNOLOGY INC
4,COGNIZANT TECHNOLOGY SOLUTIONS,COGNIZANT TECHNOLOGY SOLUTIONS US CORP
...,...,...
448,IBM,IBM CORP
449,IBM,IBM INDIA
450,IBM,IBM
451,IBM,IB SYSTEMS INC


In [22]:
#Normalize sponsors'names
match = pd.DataFrame(match)
match.rename(columns={'EMPLOYER_NAME':'Normalized Name'},inplace=True)
match.head()

,Normalized Name,Emp_name
0,COGNIZANT TECHNOLOGY SOLUTIONS,COGNIZANT TECHNOLOGY SOLUTIONS U.S. CORPORATION
1,COGNIZANT TECHNOLOGY SOLUTIONS,DB GLOBAL TECHNOLOGY INC.
2,COGNIZANT TECHNOLOGY SOLUTIONS,CLOUDMINDS TECHNOLOGY INC.
3,COGNIZANT TECHNOLOGY SOLUTIONS,CONSOLE TECHNOLOGY INC
4,COGNIZANT TECHNOLOGY SOLUTIONS,COGNIZANT TECHNOLOGY SOLUTIONS US CORP


In [23]:
#create a new dataframe

Amazon = fnmatch.filter(match[match['Normalized Name'] == 'AMAZONCOM SERVICES']['Emp_name'],'AMAZON*')
Accenture = match[match['Normalized Name'] == 'ACCENTURE'].iloc[0,:].tolist()
CTS = match[match['Normalized Name'] == 'COGNIZANT TECHNOLOGY SOLUTIONS '].iloc[0,:].tolist()
CTS1 = match[match['Normalized Name'] == 'COGNIZANT TECHNOLOGY SOLUTIONS '].iloc[4,:].tolist()
Deloitte = fnmatch.filter(match[match['Normalized Name'] == 'DELOITTE CONSULTING']['Emp_name'],'DELOITTE*')
GOOGLE = fnmatch.filter(match[match['Normalized Name'] == 'GOOGLE']['Emp_name'],'GOOGLE*')
INFOSYS = match[match['Normalized Name'] == 'INFOSYS'].iloc[4,:].tolist()
TATA = match[match['Normalized Name'] == 'TATA CONSULTANCY SERVICES'].iloc[6,:].tolist()
CAPGEMINI = fnmatch.filter(match[match['Normalized Name'] == 'CAPGEMINI ']['Emp_name'],'CAPGEMINI*')
MICROSOFT = fnmatch.filter(match[match['Normalized Name'] == 'MICROSOFT']['Emp_name'],'MICROSOFT*')
IBM = fnmatch.filter(match[match['Normalized Name'] == 'IBM']['Emp_name'],'IBM*')
TOP10 = Amazon+Accenture+CTS+CTS1+Deloitte+GOOGLE+INFOSYS+TATA+CAPGEMINI+MICROSOFT+IBM
TOP10 = pd.DataFrame(TOP10)

TOP10.rename(columns={0:'EMPLOYER_NAME'},inplace=True)
TOP10['Normalized_Name'] = 'AMAZON'
TOP10.iloc[25:27,1] = 'ACCENTURE'
TOP10.iloc[27:31,1] ='COGNIZANT'
TOP10.iloc[31:38,1] ='DELOITTE'
TOP10.iloc[38:46,1] ='GOOGLE'
TOP10.iloc[46:48,1] ='INFOSYS'
TOP10.iloc[48:50,1]='TATA'
TOP10.iloc[50:57,1] ='CAPGEMINI'
TOP10.iloc[57:60,1]='MICROSOFT '
TOP10.iloc[60:,1]='IBM'
TOP10.head()

,EMPLOYER_NAME,Normalized_Name
0,"AMAZON.COM SERVICES, INC.",AMAZON
1,"AMAZON WEB SERVICES, INC.",AMAZON
2,AMAZON ROBOTICS LLC,AMAZON
3,"AMAZON.COM SERVICES, INC",AMAZON
4,"AMAZON.COM SERVICES, INC.",AMAZON


In [24]:
#get number of petitions for top 10 sponsors
#df6 contains IT jobs data only
r = df6.merge(TOP10,on=['EMPLOYER_NAME'],how='inner')
emp_count = pd.DataFrame(r.groupby('Normalized_Name')['EMPLOYER_NAME'].count())
emp_count.reset_index(inplace=True)
emp_count.rename(columns={'EMPLOYER_NAME':'Num of H1B Petitions'},inplace=True)
emp_count

,Normalized_Name,Num of H1B Petitions
0,ACCENTURE,5295
1,AMAZON,7153
2,CAPGEMINI,8189
3,COGNIZANT,27626
4,DELOITTE,7699
5,GOOGLE,8302
6,IBM,4878
7,INFOSYS,20886
8,MICROSOFT,5078
9,TATA,11772


In [25]:
#calculate previling wages 
df8 = df2[df2['SOC_CODE'].str.contains(search_code)]
df8 = df8.dropna(subset = ['PREVAILING_WAGE_1','PW_UNIT_OF_PAY_1'])
df8.isnull().sum(axis=0)

CASE_STATUS           0
VISA_CLASS            0
JOB_TITLE             0
SOC_CODE              0
SOC_TITLE             0
FULL_TIME_POSITION    0
EMPLOYER_NAME         5
WORKSITE_STATE_1      2
PREVAILING_WAGE_1     0
PW_UNIT_OF_PAY_1      0
dtype: int64

In [26]:
#Salary comparison for top IT jobs out of 30 titles (12 titles)
Top_IT_Title = fnmatch.filter(H1B_By_Titles['SOC_CODE'].unique(),'15-*')
df9 = df8[df8['SOC_CODE'].isin(Top_IT_Title)]
#update col 'PW_UNIT_PAY_1' with data Hour and wages greater than 1000
df9 = df9.copy()
df9.loc[(df9.PREVAILING_WAGE_1 > 1000) & (df9.PW_UNIT_OF_PAY_1 == 'Hour'),'PW_UNIT_OF_PAY_1']='Year'
df9.loc[(df9.PREVAILING_WAGE_1 > 10000) & (df9.PW_UNIT_OF_PAY_1 == 'Month'),'PW_UNIT_OF_PAY_1']='Year'

In [27]:
#calculate year salary
W = df9[df9['PW_UNIT_OF_PAY_1'] == 'Week']
S = (W['SOC_CODE'], W['SOC_TITLE'],W['CASE_STATUS'],W['EMPLOYER_NAME'],W['PREVAILING_WAGE_1']*48,W['FULL_TIME_POSITION'])
result_Week = pd.DataFrame(list(S))
s_Week = result_Week.T

W = df9[df9['PW_UNIT_OF_PAY_1'] == 'Month']
S = (W['SOC_CODE'],W['SOC_TITLE'],W['CASE_STATUS'],W['EMPLOYER_NAME'],W['PREVAILING_WAGE_1']*12,W['FULL_TIME_POSITION'])
result_Month = pd.DataFrame(list(S))
s_Month = result_Month.T

W = df9[df9['PW_UNIT_OF_PAY_1'] == 'Bi-Weekly']
S = (W['SOC_CODE'],W['SOC_TITLE'],W['CASE_STATUS'],W['EMPLOYER_NAME'],W['PREVAILING_WAGE_1']*96,W['FULL_TIME_POSITION'])
result_Biweekly = pd.DataFrame(list(S))
s_Biiweekly = result_Biweekly.T 

Hour = df9[df9['PW_UNIT_OF_PAY_1'] == 'Hour']
S = (W['SOC_CODE'],Hour['SOC_TITLE'],Hour['CASE_STATUS'],Hour['EMPLOYER_NAME'],Hour['PREVAILING_WAGE_1']*1920,W['FULL_TIME_POSITION'])
result_Hour = pd.DataFrame(list(S))
s_Hour = result_Hour.T

W = df9[df9['PW_UNIT_OF_PAY_1'] == 'Year']
s_Year = W[['SOC_CODE','SOC_TITLE','CASE_STATUS','EMPLOYER_NAME','PREVAILING_WAGE_1','FULL_TIME_POSITION']]



In [28]:
#combine above data frames -- Year salary for each poistion and employer
frames = [s_Week,s_Month,s_Hour,s_Year]
Year_Salary = pd.concat(frames)
Year_Salary[Year_Salary['CASE_STATUS'] == 'DENIED']

,SOC_CODE,SOC_TITLE,CASE_STATUS,EMPLOYER_NAME,PREVAILING_WAGE_1,FULL_TIME_POSITION
343967,15-1132,"SOFTWARE DEVELOPERS, APPLICATIONS",DENIED,ALL TO SUCCESS INC.,94512,Y
169246,15-1131,COMPUTER PROGRAMMERS,DENIED,YITU INC,101424,Y
170886,15-1131,COMPUTER PROGRAMMERS,DENIED,ADMINISTRATIVE OFFICE OF THE COURTS;TN SUPREME...,67656,Y
480149,15-1199,"COMPUTER OCCUPATIONS, ALL OTHER",DENIED,ADMINISTRATIVE OFFICE OF THE COURTS;TN SUPREME...,66000,Y
93479,NaN,COMPUTER SYSTEMS ANALYSTS,DENIED,XCELTECH INC,72902.4,NaN
...,...,...,...,...,...,...
534691,15-2041,STATISTICIANS,DENIED,"ABS GLOBAL, INC",74901,Y
535073,15-2041,STATISTICIANS,DENIED,"NEUSTAR INFORMATION SERVICES, INC.",89918,Y
535141,15-2041,STATISTICIANS,DENIED,"NEUSTAR INFORMATION SERVICES, INC.",89918,Y
535719,15-2041,STATISTICIANS,DENIED,KABBAGE INC.,97906,Y


In [29]:
#for each position, calculate the avg salary 
Year_Salary[['PREVAILING_WAGE_1']] = Year_Salary[['PREVAILING_WAGE_1']].apply(pd.to_numeric)
Avg = Year_Salary.groupby('SOC_CODE')['PREVAILING_WAGE_1'].mean()
Avg = pd.DataFrame(Avg)
Avg.reset_index(inplace=True)
Avg.columns=['SOC_CODE','H1B_AVG_SALARY']
#tech_title = (pd.DataFrame(Year_Salary.iloc[:,0:2])).drop_duplicates(subset = ['SOC_CODE'],keep = 'first', inplace = True)
#Avg = pd.merge(Avg, tech_title, how='left', on ='SOC_CODE')
Avg['H1B_Title'] = ['COMPUTER AND INFORMATION RESEARCH SCIENTISTS',
                   'COMPUTER SYSTEMS ANALYSTS',
                   'INFORMATION SECURITY ANALYSTS',
                   'COMPUTER PROGRAMMERS',
                   'SOFTWARE DEVELOPERS, APPLICATIONS',
                   'SOFTWARE DEVELOPERS, SYSTEMS SOFTWARE',
                   'DATABASE ADMINISTRATORS',
                   'NETWORK AND COMPUTER SYSTEMS ADMINISTRATORS',
                   'COMPUTER NETWORK ARCHITECTS',
                   'COMPUTER OCCUPATIONS, ALL OTHER',
                   'OPERATIONS RESEARCH ANALYSTS',
                   'STATISTICIANS']

Avg.head()


,SOC_CODE,H1B_AVG_SALARY,H1B_Title
0,15-1111,112769.413140,COMPUTER AND INFORMATION RESEARCH SCIENTISTS
1,15-1121,85730.649175,COMPUTER SYSTEMS ANALYSTS
2,15-1122,92045.179703,INFORMATION SECURITY ANALYSTS
3,15-1131,74083.980762,COMPUTER PROGRAMMERS
4,15-1132,97035.008865,"SOFTWARE DEVELOPERS, APPLICATIONS"


In [30]:
#import national avg salary data for above poisitons
National = pd.read_excel('2019 National IT Occupation Data.xlsx')
National['Occupation title']= National['Occupation title'].str.upper()
salary_comparison = Avg.merge(National.iloc[:,1:9] ,left_on='H1B_Title',right_on='Occupation title')
salary_comparison.head()

,SOC_CODE,H1B_AVG_SALARY,H1B_Title,Occupation title,Level,Employment,Employment RSE,"Employment per 1,000 jobs",Median hourly wage,Mean hourly wage,Annual mean wage
0,15-1111,112769.413140,COMPUTER AND INFORMATION RESEARCH SCIENTISTS,COMPUTER AND INFORMATION RESEARCH SCIENTISTS,detail,30780,0.036,0.210,59.06,61.28,127460
1,15-1121,85730.649175,COMPUTER SYSTEMS ANALYSTS,COMPUTER SYSTEMS ANALYSTS,detail,589060,0.010,4.011,43.71,46.23,96160
2,15-1122,92045.179703,INFORMATION SECURITY ANALYSTS,INFORMATION SECURITY ANALYSTS,detail,125570,0.018,0.855,47.95,50.10,104210
3,15-1131,74083.980762,COMPUTER PROGRAMMERS,COMPUTER PROGRAMMERS,detail,199540,0.016,1.359,41.61,44.53,92610
4,15-1132,97035.008865,"SOFTWARE DEVELOPERS, APPLICATIONS","SOFTWARE DEVELOPERS, APPLICATIONS",broad,1754750,0.007,11.947,49.20,51.44,106980


In [31]:
#get the total number of H1B peptitions for each poistions 
df11 = H1B_By_Titles[H1B_By_Titles['SOC_CODE'].isin(Top_IT_Title)]
Num_Emp_comparison = df11.merge(National.iloc[:,1:4], left_on='SOC_TITLE',right_on='Occupation title')
Num_Emp_comparison.rename({'COUNT':'Total H1B Employment','Employment':'Total National Employment'},axis=1,inplace=True)
#add percentage 

#total num of IT job employment in the United States in 2019
National_Total_Emp = Num_Emp_comparison['Total National Employment'].sum()
Num_Emp_comparison['Total'] = National_Total_Emp

In [32]:
NonH1B = Num_Emp_comparison.iloc[:,1:6]
NonH1B['Num_NonH1B'] = (NonH1B['Total National Employment']-NonH1B['Total H1B Employment'])
NonH1B=NonH1B.drop(['Total H1B Employment','Total National Employment'],axis=1)
NonH1B=NonH1B[['Num_NonH1B','SOC_TITLE','Occupation title','Level']]
NonH1B['Employment 2'] ='NonH1B'
NonH1B.rename(columns={'Num_NonH1B':'Num'},inplace=True)

a = Num_Emp_comparison.iloc[:,1:5]
a['Employment level 3'] = 'H1B'
a.rename(columns={'Total H1B Employment':'Num'},inplace=True)

frames = [a,NonH1B]
treemap = pd.concat(frames)
treemap.fillna('NonH1B',inplace=True)
treemap['Employment level1']='National'
treemap['Total']=National_Total_Emp

In [33]:
#dataframe for case status for pie chart
df12 = df2[df2['SOC_CODE'].str.contains(search_code)]
count_case_status = df12.groupby('CASE_STATUS')['CASE_STATUS'].count()
count_case_status = pd.DataFrame(count_case_status)
count_case_status.rename(columns={'CASE_STATUS':'Count'},inplace=True)
count_case_status.reset_index(inplace=True)
count_case_status['Total'] = count_case_status['Count'].sum()
Percentage = pd.DataFrame((count_case_status['Count']/count_case_status['Total']).mul(100).round(0).astype(str)+'%')
Percentage = Percentage.rename(columns={0:'P'})
IT_Case_Status = pd.concat([count_case_status,Percentage],axis=1)

In [34]:
#Rejection rate between NonIT and IT jobs
#df2 - df12 = NonIT jobs
df13 = df2[~df2.index.isin(df12.index)]
count_case_status_NonIT = df13.groupby('CASE_STATUS')['CASE_STATUS'].count()
count_case_status_NonIT = pd.DataFrame(count_case_status_NonIT)
count_case_status_NonIT.rename(columns={'CASE_STATUS':'NonIT_Count'},inplace=True)
count_case_status_NonIT.reset_index(inplace=True)
status = pd.concat([IT_Case_Status,count_case_status_NonIT],axis=1)
status = status.drop(['P'],axis=1)
status['sum'] = status['Count'] + status['NonIT_Count']
status =status.iloc[2,:]
denied = status.T
denied['p_it'] = round(denied['Count']/denied['sum']*100,0)
denied['p_nonit'] = round(denied['NonIT_Count']/denied['sum']*100,0)
denied = (pd.DataFrame(denied))
denied = denied.T


In [35]:
#deney factors (salary,Job titles)
deny= Year_Salary[Year_Salary['CASE_STATUS'] == 'DENIED']

deny = deny.copy()
deny.loc[(deny.SOC_TITLE == 'COMPUTER SYSTEMS ANALYSTS') ,'SOC_TITLE']='COMPUTER SYSTEMS ANALYST'
deny.loc[(deny.SOC_TITLE == 'SOFTWARE DEVELOPERS, APPLICATIONS') ,'SOC_TITLE']='SOFTWARE DEVELOPER, APPLICATIONS'

In [36]:
#5 Numbers comparison between denied and certified H1B of TOP 12 IT jobs
#Year_Salary.head()
#count of certified case for each salary 
C = Year_Salary[(Year_Salary['CASE_STATUS']=='CERTIFIED') & (Year_Salary['FULL_TIME_POSITION']=='Y')]
              
C_COUNT = pd.DataFrame(C.groupby('PREVAILING_WAGE_1')['CASE_STATUS'].count())
C_COUNT.reset_index(inplace=True)
C_COUNT.rename(columns={'CASE_STATUS':'COUNT'},inplace=True)
C_COUNT['STATUS'] ='CERTIFIED'

#count of denied case for each salary 
D = Year_Salary[(Year_Salary['CASE_STATUS']=='DENIED') & (Year_Salary['FULL_TIME_POSITION']=='Y')]
D_COUNT = pd.DataFrame(D.groupby('PREVAILING_WAGE_1')['CASE_STATUS'].count())
D_COUNT.reset_index(inplace=True)
D_COUNT.rename(columns={'CASE_STATUS':'COUNT'},inplace=True)
D_COUNT['STATUS'] ='DENIED'
#D_COUNT

F = [C_COUNT,D_COUNT]
Box = pd.concat(F)

Box.head()


,PREVAILING_WAGE_1,COUNT,STATUS
0,17410.0,3,CERTIFIED
1,18699.0,4,CERTIFIED
2,19594.0,2,CERTIFIED
3,23941.0,1,CERTIFIED
4,24128.0,1,CERTIFIED


In [37]:
#the following code for IT jobs only
#get total filed H1b
last = Year_Salary.merge(TOP10,on=['EMPLOYER_NAME'],how='inner')
last_filed = pd.DataFrame(last.groupby('Normalized_Name')['CASE_STATUS'].count())
last_filed.reset_index(inplace=True)
last_filed.columns=['Normalized_Name','Total Filed H1B']
#last_filed

#get total certified cases
final = last[last['CASE_STATUS']=='CERTIFIED']
last_pass = pd.DataFrame(final.groupby('Normalized_Name')['CASE_STATUS'].count())
last_pass.reset_index(inplace=True)
last_pass.columns=['Normalized_Name','Total Certified H1B']
#last_pass

#get certified rate
#merge above two dfs first
combined = last_filed.merge(last_pass,on=['Normalized_Name'],how='inner')
combined['Certified_Rate'] = round(combined['Total Certified H1B'] / combined['Total Filed H1B'] *100 ,2)
#combined

#get the salary mean
wage = pd.DataFrame(round(final.groupby('Normalized_Name')['PREVAILING_WAGE_1'].mean(),2))
wage.reset_index(inplace=True)
wage.columns=['Normalized_Name','Avg Salary']
combined1 = combined.merge(wage,on=['Normalized_Name'],how='inner')
combined1

,Normalized_Name,Total Filed H1B,Total Certified H1B,Certified_Rate,Avg Salary
0,ACCENTURE,5228,5119,97.92,87719.23
1,AMAZON,7065,6939,98.22,99304.57
2,CAPGEMINI,8131,8004,98.44,86421.20
3,COGNIZANT,27181,26898,98.96,86142.18
4,DELOITTE,7478,7369,98.54,85183.25
5,GOOGLE,7522,7167,95.28,115424.03
6,IBM,4680,3964,84.70,89259.83
7,INFOSYS,20767,20754,99.94,80728.88
8,MICROSOFT,4975,4975,100.00,114618.13
9,TATA,10082,10058,99.76,72351.87


In [38]:
d = pd.DataFrame(final.groupby('PREVAILING_WAGE_1')['CASE_STATUS'].count())
d.reset_index(inplace=True)
d.columns=['Salary','Count of Ceritified Cases']
d

,Salary,Count of Ceritified Cases
0,18699.0,4
1,19594.0,2
2,24544.0,1
3,25542.0,1
4,32766.0,1
...,...,...
3018,176072.0,1
3019,179150.0,1
3020,180294.0,1
3021,184350.0,3


# Visualization

### Number of Petitions - Top 30 Job Titles 

In [79]:
#pyo.offline.init_notebook_mode(connected=True)
fig = px.bar(H1B_By_Titles, x = 'SOC_TITLE', y = 'COUNT',
            hover_data = ['SOC_TITLE','COUNT'],
            labels  = {'SOC_TITLE':'Job Titles'})
fig.update_layout(xaxis_tickangle=-45,
                 width = 1000, height=700,
                 title_text = 'Top 30 Jobs with H1B petitions in 2019')
#fig.show()
#py.iplot(fig, filename='Top30Jobs')
pyo.offline.plot(fig,filename='Top30Jobs')
#pio.write_html(fig,file='Top30JOBS.htlml',auto_open=False)

'Top30Jobs.html'

### H1B IT Job Locations

In [56]:
fig = go.Figure(data = go.Choropleth(locations=df5['state'],
                                   z = df5['count'] , 
              locationmode = 'USA-states',
               colorscale = 'Reds',             
               colorbar_title = 'Number of H1B petitions'))
fig.update_layout(geo_scope = 'usa',
                 title_text = 'Number of H1B petitions of IT jobs by states ')
fig.show()

### Top 10 H1B Sponsors

In [101]:
fig = px.bar(emp_count,x='Num of H1B Petitions',y='Normalized_Name',
          orientation='h' ,color='Normalized_Name' )
fig.update_layout(title_text='Top 10 Companies filed most H1B petitions in 2019',showlegend=False)
fig.show()

### Number of employment of H1B vs Nationals by Top 12 IT Positions

In [102]:
fig = px.treemap(treemap, path=['Total','Occupation title','Employment level1','Employment level 3']
                , values = 'Num')
fig.update_layout(title_text='H1B vs Nationals in Number of Employment In 2019')
fig.show()

### Top 12 IT jobs Avg Salary between H1B and National 

In [103]:
job = salary_comparison['H1B_Title']
fig = go.Figure(data=[
    go.Bar(name='H1B Avg Salary',x=job,y=salary_comparison['H1B_AVG_SALARY']),
    go.Bar(name='National Avg Salary',x=job, y=salary_comparison['Annual mean wage'])
    ])

fig.update_layout(title_text='Top 12 IT Positions Salary Comparison between H1B and National In 2019')

fig.show()

### Percentage by Case Status

In [104]:
labels = ['Certified','Certified-Withdrawn','Denied','Withdrawn']
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels,values=[90,6,1,3],name='IT Jobs'),
             1,1)

fig.add_trace(go.Pie(labels=['IT','Non-IT'],values=[48,52],name='Deneid'),
                    1,2)

fig.update_layout(
        title_text = "Case Status Rates",
        annotations=[dict(text='IT Jobs', x=0.18, y=0.5, font_size=19, showarrow=False),
                 dict(text='Deneid Rate', x=0.87, y=0.5, font_size=19, showarrow=False)])

fig.update_traces(hole=.5,hoverinfo='label+percent+name')

fig.show()

### What may casuses H1B deneid?

#### A. Denied H1B by Salary and Job titles

In [105]:
job = deny['SOC_TITLE']
wage = deny['PREVAILING_WAGE_1']
fig = px.scatter(deny,x=job,y=wage,
                color=job
                )
fig.update_xaxes(showticklabels=False)
fig.update_layout(title_text='Denied H1B by IT Positions and Salaries',
                 xaxis_title=  'IT Positions',
                 yaxis_title='Salary')
fig.show()

#### B. 5 numbers comparison between Certified and Denied H1B (IT Jobs FULL TIME ONLY)

In [106]:
fig = px.box(Box,x='STATUS',y='PREVAILING_WAGE_1',color='STATUS')

fig.update_layout(title_text='5 numbers comparison of Certified & Denied Full Time H1B visa (IT ONLY)',
                  yaxis_title=  'SALARY')
fig.show()

### The company pays least but with highest certified rate

In [107]:
fig = px.scatter(combined1,x='Certified_Rate',
                y='Avg Salary',
                size='Avg Salary',
                color='Normalized_Name')
fig.update_layout(title_text='Top10 Sponsors - Certified Rate vs Avg Salary')
fig.show()

### Does salary impacts certified rate? (IT Jobs ONLY)

In [108]:
g=d['Count of Ceritified Cases']
u=d['Salary']
fig = go.Figure(data=go.Scatter(x=u,
                            y=g,
                            mode='markers'))
fig.update_layout(title_text='Does salary impacts certified rate?',
                 yaxis_title=  'Number of Certified H1B',
                 xaxis_title ='Salary')
fig.show()

# Conclusion

##### In 2019, more than half of H1B cases filed with IT positions. In the top 30 job titles, IT jobs occupy 12 out of it. The title 'software developers for applications' has more than 200k H1B cases. Based on the map, most IT jobs are located on the East, South, and West coasts. California has the most IT jobs in the States, which has at least 86k. For those 12 IT jobs, Cognizant sponsored more than 25k cases in 2019. In other words, Cognizant applied 50% of the H1B visa in the IT field.  In comparing the number of employments, H1B workers have different impacts on the nation with different positions. But for each IT job, the average salary of H1B workers is lower than the national average. Among all IT jobs, the denied rate of H1B is only 1%, but compared to Non-IT jobs, most rejected cases are from the IT industry. Salary has minor influences on the H1B certified rate. Still, it does not have a linear relationship with the certified rate, which means a high salary does not guarantee to get an H1B visa certified. 
